In [1]:
import numpy as np

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR

import torchvision

from tqdm.notebook import tqdm

import train as training

In [2]:
dire = "./cleaned-data" # directory of dataset
# loading data loader
trainloader, testloader = training.preprocess_data(directory=dire, batch_size=64, test_size=0.3, rand_num=40, worker=2)

{0: 426, 1: 426}


In [3]:
# load pretrained model
# vgg16 = torchvision.models.vgg16(pretrained=True)
# googlenet = torchvision.models.googlenet(pretrained=True)
# efficientnet_b7 = torchvision.models.efficientnet_b7(pretrained=True)
resnet50 = torchvision.models.resnet50(pretrained=True)

In [4]:
class ClassifierNet(nn.Module):
    def __init__(self, pretrained: nn.Module):
        super().__init__()
        # Pretrained
        self.network = pretrained
        # Replace last layer
        self.network.fc = nn.Sequential(nn.Linear(2048, 512), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.25),
                                         nn.Linear(512, 128), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.50), 
                                         nn.Linear(128, 2),
                                         nn.Softmax()
                                        )
    def forward(self, x):
        out = self.network(x)
        return out


In [5]:
# define params here
net = ClassifierNet(resnet50)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.9)

criterion = nn.CrossEntropyLoss()
epochs = 50
training.run(net, optimizer, scheduler, criterion, epochs, trainloader, testloader)

Training on cuda


  0%|          | 0/50 [00:00<?, ?it/s]

C:\Users\goodu\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/50], Step [5/38], Accuracy: 51.250, Train Loss: 0.6737
Epoch [1/50], Step [10/38], Accuracy: 48.750, Train Loss: 0.7080
Epoch [1/50], Step [15/38], Accuracy: 50.833, Train Loss: 0.6953
Epoch [1/50], Step [20/38], Accuracy: 52.188, Train Loss: 0.6821
Epoch [1/50], Step [25/38], Accuracy: 52.250, Train Loss: 0.6801
Epoch [1/50], Step [30/38], Accuracy: 53.125, Train Loss: 0.6608
Epoch [1/50], Step [35/38], Accuracy: 53.214, Train Loss: 0.7300
-----------------------------------------------
Epoch [2/50], Step [5/38], Accuracy: 67.500, Train Loss: 0.6706
Epoch [2/50], Step [10/38], Accuracy: 68.125, Train Loss: 0.6444
Epoch [2/50], Step [15/38], Accuracy: 69.583, Train Loss: 0.5849
Epoch [2/50], Step [20/38], Accuracy: 69.688, Train Loss: 0.5552
Epoch [2/50], Step [25/38], Accuracy: 71.000, Train Loss: 0.5149
Epoch [2/50], Step [30/38], Accuracy: 71.458, Train Loss: 0.5078
Epoch [2/50], Step [35/38], Accuracy: 71.250, Train Loss: 0.6589
--------------------------------------------

KeyboardInterrupt: 